# TODO
Test in another folder that the zip send will work

# Installation
## Cytomine:
Using pip, you can do it with these two lines:
~~~
$ curl -s https://packagecloud.io/install/repositories/cytomine-uliege/Cytomine-python-client/script.python.sh | bash
$ pip install cytomine-python-client
~~~


# Import


In [ ]:
from cytomine import Cytomine
from cytomine.models import ProjectCollection, ImageInstanceCollection, AnnotationCollection
import getpass
import shapely
from shapely import wkt
from shapely.affinity import affine_transform
import cv2                             # OpenCV
from imgTools import display, multiDisplay
import os, sys
import logging
import segment_detector as sd
import numpy as np
from metrics_lines import metricNaive, metricFalseNeg, metricFalsePos, metricTot
from images_database.preprocess_eyes import img_eye_partial_preprocessing
from images_database.preprocess_soccer import preprocessSoccerImage
from images_database.augment_data_set_soccers import report_augmented_image_and_annotations

# Image database
The evaluation requires the database with all images. If you want to run the evaluation, please provide us the path to the folder(s) containing the images with the code bellow.

In [ ]:
resp = input("Do you want to run the evaluation? (If not, what follows will not be needed) [y, n] : ")
if resp == 'n':
    EVALUATE = False
    PATH_FOLDER_IMG = None
else:
    EVALUATE = True
    print("Could you please provide us the path to the folder?")
    PATH_FOLDER_IMG = []
    while True:
        print("\nThe current list of folders is : ")
        for path in PATH_FOLDER_IMG:
            print(f'\t- {path}')
        resp = input("Do you want to add a new folder? [y, n] : ")
        if resp == 'n':
            break
        else:
            resp = input("What is the new path? : ")
            PATH_FOLDER_IMG.append(resp)

if EVALUATE:
    print("\n\nThe final list of folders is : ")
    for path in PATH_FOLDER_IMG:
        print(f'\t- {path}')

# Cytomine connection
This is used to get the line annotations.

In [ ]:
if EVALUATE:
    host = 'https://learn.cytomine.be'
    public_key = getpass.getpass('Public key of cytomine account : ')   # u can get it on your account
    private_key = getpass.getpass('Private key of cytomine account : ') # u can get it on your account

    CONN = Cytomine.connect(host, public_key, private_key, verbose=logging.ERROR)
    # Check https://docs.python.org/3.6/library/logging.html#logging-levels  to see other verbose level
    print(CONN.current_user)

# Evaluation of line detection
## Metrics description
To evaluate the line detection previously implemented, it was required to choose a metric. The metrics  for segment detection were designed from scratch as no satisfying metrics were found during our research.

<br><br>

**From annoted line parameters to annoted images**

The images of the annoted line is obtained by drawing the line with a width of 3 pixels on a black image of the same size. The line width of 3 was chosen arbitrarily to take into consideration the degree of precision of the annotation which is not perfect.

<br><br>

**Positive and negative classification**

Let's define some important terms : 
$$
\begin{array}{ll}
      \text{line pixel } &= \text{ pixel classified as a line by the annotation}\\
      \text{not a line pixel } &= \text{ pixel classified as not a line by the annotation}\\
      \text{positive pixel } &= \text{ pixel classified as a line by the dectection}\\
      \text{negative pixel } &= \text{ pixel classified as not a line by the dectection}\\
      \text{true positive pixel } &= \text{ line pixel classified as a line by the dectection}\\
      \text{false positive pixel } &= \text{ not a line pixel classified as a line by the dectection}\\
      \text{true negative pixel } &= \text{ not a line pixel classified as not a line by the dectection}\\
      \text{false negative pixel } &= \text{ line pixel classified as not a line by the dectection}\\
\end{array}
$$

<br><br>

**Naive metric**

The first implemented metric is :
$$\text{metric naive } = \frac{p_c}{p_t} \quad \text{with} \left\{
    \begin{array}{l}
      p_c = \text{ number of pixels well classified}\\
      p_t = \text{ total number of pixels}\\
    \end{array}
\right.$$

The metric give a value of the accuracy in the interval [0,1].

As the name indicates, it is a very naive metric as the number of line pixels are so low that an image with no lines detected will give a good result.

<br><br>

**False negative metric**

The second implemented metric is based on penalization of the false negative pixels :
$$\text{metric false neg. } = min\big(1, \frac{1}{a_{neg}}\frac{p_{tp}}{p_l}\big) \quad \text{with} \left\{
    \begin{array}{l}
      p_{tp} = \text{ number of true positive pixels }\\
      p_{l} = \text{ total number of line pixels}\\
      a_{neg} = \text{ acceptable accuracy for false negative}\\
    \end{array}
\right.$$
The acceptable accuracy is arbitrarly fixed at $a_{neg} = 0.8$. As the annotation are not perfectly made due to human error, a perfect accuracy should not be expected with this metric either. Thus, using this accepetable metric will simply give the best accuracy score for $\frac{p_{tp}}{p_l}\geq 0.8 $.

The metric give a value of the accuracy in the interval [0,1].

This metrics has the disadvantage of giving a perfect score for a method detecting each pixels as a line. Thus, a new metric is introduced : the false positive metric.

<br><br>

**False positive metric**

This third metric is based on penalization of the false positive pixels:
$$\text{metric false pos. } = min\big(1, \frac{1}{a_{pos}}(max(0, 1-\frac{3\;p_{fp}}{p_{nl}}))\big) \quad \text{with} \left\{
    \begin{array}{l}
      p_{fp} = \text{ number of false positive pixels }\\
      p_{nl} = \text{ total number of not a line pixels}\\
      a_{pos} = \text{ acceptable accuracy for false positive}\\
    \end{array}
\right.$$
As $p_{nl}$ is usually a high number, this metric can easily give a high score, thus the acceptable accuracy for false positive is fixed to $a_{pos} = 1$ and the number of false positive is multiplied by 3 to have a higher penalization.

The metric give a value of the accuracy in the interval [0,1].

This metric has the opposite disadvantage of the false negative metric, a method detecting no line on the image will always give a perfect score. 

<br><br>

**Total metric**

As the two previous metric had opposite disadvantage, it is easy to convince ourself that combining the two will obtain a new metric robust to those two cases.

$$\text{metric tot. } = w_1\;\text{metric false neg. } + w_2\;\text{metric false pos. }\quad \text{with} \left\{
    \begin{array}{l}
      w_1 \in [0,1]\\
      w_2 \in [0,1]\\
      w_1 + w_2 = 1\\
    \end{array}
\right.$$

The metric give a value of the accuracy in the interval [0,1]. The weight were both set to $w_1 = w_2 = 0.5$ such that both the method detecting lines on all pixels and the detecting no lines would have a score of about $0.5$.

Note that the false positive metric is very simple and can easily give high score, thus changing it to a better one would give more representative results of the detection method accuracy.


## Example line metrics

In this example, the metric for one image is given for 4 cases with the corresponding scores:

$
\left.
    \begin{array}{ll}
      \quad\text{- the actual detection } &: m_{tot} \simeq 0.7\\
      \quad\text{- all pixels as not a line } &: m_{tot} \simeq 0.5\\
      \quad\text{- all pixels as line} &: m_{tot} \simeq 0.5\\
      \quad\text{- all pixels randomly detected as line or not } &: m_{tot} \simeq 0.3\\
    \end{array}
\right.$

The score are as expected much better for the actual detection than the other ones. It can be noticed that the false positive metric is quite harsh. It gave a low score to the actual detection and a score of 0 to the randomly pixels. This will cause the total metric to only give good score for image with high number of true positive and very low number of false positive.

In [ ]:
import metrics_line_example

metrics_line_example.evaluate(CONN)        

## Evaluation of the database
Using the metrics described above to evaluate our detection method on the full database will give the corresponding scores:


$
\left.
    \begin{array}{ll}
      \quad\text{- metric naive } &: m_{nai} = 0.945\\
      \quad\text{- metric false neg. } &: m_{neg} = 0.725\\
      \quad\text{- metric false pos. } &: m_{pos} = 0.860\\
      \quad\text{- metric total } &: m_{tot} = 0.793\\
    \end{array}
\right.$

The false negative metric has a good accuracy, however, it still has room for improvement. The false positive metric is very high which will improve the total metric accuracy. As explained earlier, the false positive metric is simple and easily give high score, even though it is not that good to the human eye. However it was difficult to find another metric to penalize correctly the false positive pixels.

In general, the evaluation indicates that our detection method has good accuracy but could have been better.

In [ ]:
import evaluateSegDetec

if EVALUATE:
    resp = input("Do you want to evaluate the line detector of part1? \nThis will takes 2-3 minutes for the full database. [y, n] : ")
    if resp == 'y':
        evaluateSegDetec.evaluate(PATH_FOLDER_IMG, CONN)

# Evaluation of ellipse detection
## Metrics description
### Distance between two ellipses
It is assumed that the ellipses are defined by 5 parameters:
- Xc : the vert. coord of the center of the ellipse
- Yc : the horiz. coord of the center of the ellipse
- theta : the angle of the main axis
- a  : half the length of main axis
- b  : half the length of sub axis

~~~
elps1 = [Xc1, Yc1, theta1, a1, b1]
elps2 = [Xc2, Yc2, theta2, a2, b2]
W = [Wx, Wy, Wt, Wa, Wb]
dist = Wx * abs(Xc1-Xc2) + Wy * abs(Yc1-Yc2) + Wt * min(abs(theta1-theta2), abs(180-abs(theta1-theta2))  + Wa * abs(a1-a2) + Wb * abs(b1-b2)
~~~
The weight W is tuned such that the maximal acceptable error on each parameters will lead to an increase in the distance of 1. Thus if we want an acceptable error of theta of 15 degrees, we fix Wt = 1/15. This way, we can fix the tresh_dist = 5 (see next subsection). This is the maximal acceptable distance between two ellipses. By fixing it at 5, we only need to tune W. The parameters W will depend on the precision expected on the labelled ellipse. 

Note that as a difference of 180 degree in theta correspond to the same angle, 'min(abs(theta1-theta2), abs(180-abs(theta1-theta2))' is used instead of 'abs(theta1-theta2)'.

In practice, the weights used are : $$W = [\frac{1}{2}, \frac{1}{2}, \frac{1}{5}, \frac{1}{3}, \frac{1}{3}]$$

Example : If we assume that the label for the x coord. of the center is has a precision of two pixels in, i.e there is a two pixels difference between the true x coord and the x coord. labelled, then we can put Wx = 1/2.

### Metric on multiple images
Let ground_truth and detected both being a list of list of ellipses, respectivelly being the labeled ellipses for each images and the detected ellipses for each images. The evaluation is done on the following pseudo-code:

~~~
metrics_elps(ground_truth, detected, tresh_dist):
    eval = 0
    for each image:
        gt_elps = ground_truth of the corresponding image
        dt_elps = detected of the corresponding image

        num = 0
        denom = max(len(gt_elps), len(dt_elps))

        dists = np.zeros((len(gt_elps), len(dt_elps)))
        for el in dists:
            el = dist(gt_elps[index1(el)], dt_elps[index2(el)]

        index1_rem = []
        index2_rem = []
        for i in range(min(len(gt_elps), len(dt_elps))):
            el = min(dists) where (not index1(el) in index1_rem) and (not index2(el) in index2_rem)
            index1_rem.append(index1(el))
            index2_rem.append(index2(el))

            num += min(1, tresh_dist/el)

        eval += num/denom
    return eval/num_image
~~~

In other words, the evaluation metric give a value between 0 and 1, 1 being the best accuracy. The evaluation for multiple images is the mean of all image evaluations. For each image, the evaluation of this images is a fraction. Let $$f(x) = min(1, \frac{tresh\_dist}{x}),$$ a function which returns 1 if x < tresh_dist and something < 1 in other cases. The choice of the tresh_dist is done such that a distance lower than it doesn't make sense compared to the precision of the labelling.

The numerator is the sum of function f on the smallest distances between the detected ellipses and the labeled ellipses where each detected and labeled ellipse can only be used once. When the number of detected and labeled ellipses aren't the same, the distance are computed until no pair can be made.

In the other hand, the denominator is the maximum between the number of detected and labeled ellipses. Thus the value can decrease due to two reasons : if the detection detect less or more ellipses than in the ground truth and if the detected ellipses are distant from the labeled ones.


In the special case where there is 0 ellipse in the ground truth the score is 0 if ellipse are detected or 1 if no ellipse are detected.

## Example of evaluation
In the example bellow, the metric for different detection will be displayed. As explained above, the number of ellipse detected will influence the final metric value. To better understand the detection strength and weakness, the metric with or without the penalization for detecting more/less ellipse than existing ellipse will be used.

It is tested on 5 cases:
    - Same parameters with a 180 change in theta
        => metric = 1 for both
    - Two ellipses detected with wrong parameters 
        => it is seen that detecting x times more ellipse than existing ones will divide the score by x (x = 2 here).
    - Detecting an ellipse with parameters in the limit of the acceptable error
        => metric = 1 for both.
    - Detected ellipse close to the ground truth but not exactly it.
    - The ground truth detected but other too 
        => it will give a low score. 

In [ ]:
import metrics_elps_example

metrics_elps_example.evaluate()        

# Evaluation of bounding box detection
## Metrics description
### Distance between two ebounding box
The metric and distance are the same as for the ellipse. The only differences is that there are only 4 parameters (the coordinates of the bottom left corner and top right corner), the weight are $W = [\frac{1}{2},\frac{1}{2},\frac{1}{2},\frac{1}{2}]$ and the tresh_dist is now equal to 4.

## Example of evaluation
In the example bellow, the metric for different detection will be displayed. As explained above, the number of ellipse detected will influence the final metric value. To better understand the detection strength and weakness, the metric with or without the penalization for detecting more/less ellipse than existing ellipse will be used.

It is tested on 5 cases:
    - Same parameters
        => metric = 1 for both
    - Two boxes detected with wrong parameters 
        => it is seen that detecting x times more box than existing ones will divide the score by x (x = 2 here).
    - Detecting a box with parameters in the limit of the acceptable error
        => metric = 1 for both.
    - Detected box far from the ground truth.
    - The ground truth detected but other too 
        => it will give a low score. 

In [ ]:
import metrics_box_example

metrics_box_example.evaluate()        


# Conventional methods for ellipses detection

The goal here is to find ellipses in an image. The ellipses are then returned as a list of five parameters: the center position, the half lengths of the main axes and the orientation angle of the major axis. On the one hand, we can represent the ellipses and on the other hand count their number if we trust the algorithm enough. We can therefore compare this method to methods based on machine learning.

In order to solve this problem, four methods were explored but only two were selected.

## Ellipse Hough transform 

This method has been tried but not selected for further works due to slowness.

### Description
This method works like the usual hough transform. Depending on its implementation, it will take a certain number of points and accumulate the parameters of the ellipses that could pass through these points. Once all points have been tested, the accumulators are evaluated and all parameters exceeding a certain threshold are selected as good candidates to represent an ellipse. These parameters will then form the different ellipses detected by the algorithm.

### Pros
* Very robust against noise
* Very robust against partial occlusion
* Easy to implement / understand

### Cons
* Takes "age of the univers" to execute, not tractable for huge images
* Not suitable for embedded computing / live applications

## Hough transform with parameter separation

This method has been implemented up to center detection but not selected for further works due to high bias and low performance on images containing several ellipses.

### Description
This method is very similar to the previous one. Here what will be tried is to separate the imposing transform into several small transforms. The one we studied is the one presented by Le Troter et al. (Arc of ellipse detection for video image registration, 2005) . The idea is to first look for the center of the ellipses and then find the other parameters from the center. To do this, triplets of points are selected. For each of them, the most likely tangent is evaluated by filtering (a mask is created for each tangent and the one that most closely resembles the neighborhood of the point is chosen). 

Then, a property of the ellipses allows to find the center of the ellipse formed by these three points. In fact, we know that for two points, if we link the intersection of their tangent to their center, the line thus formed will pass through the center of the ellipse. The crossing of the two lines thus formed by the triplet is therefore the hypothetical centre of an ellipse. These centers are accumulated and the ones with the highest scores are chosen as actual ellipse centers.
Arc of ellipse detection for video image registration
From there, the three remaining parameters are accumulated.

### Pros
* Faster than Ellipse Hough transform
* Possibly suitable for embedded computing and live applications

### Cons
* Very sensitive to accuracy errors

## Ellipse detection by accumulation of the secondary axis

This method is fully implemented and used.

### Description

This technique follows what was proposed by Yonghong Xie and Qiang Ji (A New Efficient Ellipse Detection Method, 2002). The idea will be to consider all pairs of points as the main axis of an ellipse. Then, for all the other points, we'll pretend they're on the ellipse. From these three points (the new selected point and the ends of the major axis), the half length of the secondary axis is evaluated and accumulated. Major axes with a secondary axis whose score exceeds a certain threshold are retained as the parameters of an ellipse. As it is also possible to find the other parameters from what is known about the ellipse, the ellipse is totally known at this point. Indeed, the center of the ellipse corresponds to the center of the two points chosen as the ends of the major axis. The angle of the major axis can also be evaluated from these two points.

Two sub-implementations have been used here. The first is to remove the pixels from the image once an ellipse has been detected (this avoids duplicates). The other implementation keeps all ellipses but only returns the one that received the most votes. The second method has the advantage of being more accurate but it is also slower and does not detect more than one ellipse.

### Pros
* Way faster than Ellipse Hough transform
* Suitable for embedded computing and live applications

### Cons
* Sensitive to noise (as an example, this method doesn't work well on soccer images due to the lines. A solution would be to first remove the lines here).

### Demo

The demonstration uses our implementation of the Yonghong Xie and Qiang Ji algorithm with pixel removal. The algorithm has 6 parameters which are the minimum and maximum half lengths of the two main axes, the number of bags in the accumulator and the threshold. The images used are only from the eye dataset because it takes a long time to find the ellipses in soccer images as their 'white pixel / black pixel' ratio is higher. Also, the algorithm was implemented in python and not C++ or full numpy which tends to slow down the computation.


In [ ]:
import classical as cl

original_eye = cv2.imread("./ReportImages/elps_eye01_2014-11-26_08-54-45-007.png", cv2.IMREAD_GRAYSCALE)
preprocess_eye = img_eye_partial_preprocessing(original_eye)

imageBinary = cv2.Canny(preprocess_eye, 20, 100)

ellipses = cl.youghongQiangEllipse(imageBinary, 15, 55, 5, 25, 40, 10, onlyBest=False, ratio=0.5)

imageBGR = cv2.cvtColor(original_eye, cv2.COLOR_GRAY2BGR)
for ell in ellipses:
    cv2.ellipse(imageBGR, ell[0], ell[1], ell[2], 0, 360, (0, 0, 255))
    
multiDisplay(["Pre-filtered image", "Result"], [imageBinary, imageBGR], 2)

Sometimes, the algorithm in this configuration detects 0 or more ellipses, not always a single one. This is why the "onlyBest" option is there to output only the best ellipse and not all those exceeding a certain threshold. However, the execution of this version of the algorithm is much slower. The reader can try it by modifying the "onlyBest" argument of the yongQiangEllipse function but the test is not performed by default in the notebook.  

## Evaluation of the ellipse detection by accumulation of the secondary axis on the database
Using the metrics described above to evaluate our detection method on the full database will give the corresponding scores:

- On the eye images with 1 ellipse
$$
\left.
    \begin{array}{ll}
      \quad\text{- metric penalizing } &: m_{p} = 0.36\\
      \quad\text{- metric non penalizing } &: m_{np} = 0.58\\
    \end{array}
\right.$$

- On the eye images with 0 ellipse
$$
\left.
    \begin{array}{ll}
      \quad\text{- metric penalizing } &: m_{p} = 0.21\\
      \quad\text{- metric non penalizing } &: m_{np} = 0.21\\
    \end{array}
\right.$$

In the case of the images with 1 ellipse, the average number of ellipses detected is : $\frac{0.58}{0.36} = 1.61$ which is pretty good. However, in the case of 0 ellipse on the image, only 1 image out of 5 did the algorithm detect no ellipse which isn't that good. Going back to the case of 1 ellipse, the metric non penalizing reach a good accuracy.

Giving our weights, assuming that the error in split equally on the parameters, the ellipse detected will have a distance to the ground truth ellipse of : 
$$\frac{2}{0.6} = 3.33\text{ pixels of Xc}, 3.33\text{ pixels of Yc}, 8.33\text{ degrees of theta},5 \text{ pixels of a},5 \text{ pixels of b}  $$

In conclusion, the classical method has a good accuracy but struggle to detect the exact number of ellipses.

In [ ]:
import evaluateElpsClassical

if EVALUATE:
    resp = input("Do you want to evaluate the line detector of part1? \nThis will takes several minutes for the full database. [y, n] : ")
    if resp == 'y':
        evaluateElpsClassical.main(PATH_FOLDER_IMG)

# Preprocessing

The processing for the eyes and the soccer image dataset applied is different because of the different objective aims for each of them.

## Eyes

The aim for this dataset is to isolate the iris of the eye. This is done through multiple steps. The particularity of the iris is that it contains the darkest pixel of the images. So the main procedure will rely on increasing the contrast of the image. 

First, we proceed to a normalisation of the gray-scale image. Then we increase the contrast by multiplying all the grayscale values by a given factor. The factor has been chosen according to the value histogram of the whole dataset with some manual tuning. We had chosen to use a 15 factor because it was a good trade-off for isolating the iris and reduce the negative impact of some group of images with eye-liners which are of the same level of grey or darker than the iris.

Then, we apply a binary threshold by applying the Otsu algorithm to define the threshold. It succeeds globally in splitting the value histogram between the skin/rest of the eye and the iris (potentially eyelashes). These two successive operations are similar to an increase in contrast.  Finally, we finish the preprocessing with dilatation to close the shape of the iris. A close operation wasn't needed because they weren't any isolated small artifact after the increase in contrast.

In [ ]:
original_eye = cv2.imread("./ReportImages/elps_eye01_2014-11-26_08-54-45-007.png", cv2.IMREAD_GRAYSCALE)
preprocess_eye = img_eye_partial_preprocessing(original_eye)
multiDisplay(["Preprocess Eye", "Original Eye"], [original_eye, preprocess_eye], 2)

## Soccer

The aim for this dataset is to isolate the line of the soccer field the best as we can. This is rather more complex than the preceding context. The end of this process was inspired from a paper from the Sharif University of Technology in Iran. "Automatic Soccer Field Line Recognition by Minimum Information" : http://confnews.um.ac.ir/images/41/conferences/aisp2015/183_3.pdf. 

The following step where apply to the image after blurring to reduce the complexity of some information and uniform the images in its different colour space.

We first take back our HSV mask approch from the first part of this project. By applying an HSV mask, we are able to catch all the pixel corresponding to the green spectrum by determining the range of the Hue value. The range was defined by the histogram and manual adjustment. This first step isolates most of the soccer field.

Then, we keep only the biggest non black 4-way connected component of the images in order to remove all artefact outside the soccer field. Before doing this, we apply a dilatation operation with a horizontal rectangle kernel. Because, when processing a image from the side of the field, the central line of the field is composed of white value corresponding to the whole Hue range and some dirt (brown Hue range) could be observed on its border in a considerable number of pixels. This issue splits the soccer field into two 4-way connected components. But the horizontal rectangle dilatation succeeds in connected these two components without connecting them to outsider artefacts. 

When this is done, we add the remaining White value missing from the first mask by applying a range on the saturation and the value, low saturation range [0, 30] and high value [190, 255]. These values were defined by observation on some case because the histogram wasn't helping significantly to identify a pic. After applying the 2nd HSV mask, we proceed again in an isolation of the biggest 4-way connected component.  

The last part is focused on retrieving the lines. The quote paper explains the approach we apply in a small section. The main idea is to proceed to a median filtering in the LAB colour space on the L channel only. The L channel represents luminescence value and is not equivalent to the Grey Scale nor the Value channel of HSV because it applies a nonlinear transformation of the RGB space. This colour space is slightly better for filtering operation because it better dissociates brightness and chromatic data. The median filtering will help to uniform the luminescence over the soccer field and its dimension were chosen to be able to drop the luminescence data from the white line. Then we subtract the image before the operation and the filtered image. After applying a binary threshold (\[0, 10\]\[11, 255\]) and remove small connected component (closing operation could have wipe out thin white lines), we have our final prepossessing image which consists of all bright pixel on the soccer field.

In [ ]:
original_soccer = cv2.imread("./ReportImages/elps_soccer01_1024.png", cv2.IMREAD_COLOR)
preprocess_soccer = preprocessSoccerImage(original_soccer)
multiDisplay(["Preprocess Soccer", "Original Soccer"], [original_soccer, preprocess_soccer], 2)

## Data Augmentation

The objective was to extend the dataset of soccer images because it is by far the most complicated case for the deep learning model. In order to get synthetic images with synthetic annotations, we based our approach on the already preprocessed dataset from the original soccer images dataset. Because the preprocessing parameters are really sensible to a certain level of zoom and we aren't sure of how the preprocess of modified images will result. It is also a good way to save resources and to get as fast as possible new data. The lines are already extracted so we applied changes to these images to simulate plausible contexts. We are using the following affine transformations:

- Horizontal Flip
- Zoom [0.8, 0.95] (Zoom in) & [1.2, 2] (Zoom out)
- Shift according to y axis  +/- [0.05 * height, 0.2 * height]

The parameter used are chosen from the previous range according to a uniform law to not bias the simulation and to get something different from the original prepossessed image. Because shifting according to x and zoom in are excluding ellipses from the bounds, we didn't apply or restraint the factor. We use this new augmented dataset according to the variation in accuracy in the model.

In [ ]:
soccer_image = cv2.imread("./ReportImages/elps_soccer01_1056.png" , cv2.IMREAD_GRAYSCALE)
flip_image = report_augmented_image_and_annotations("FLIP", soccer_image, "elps_soccer01_1056.png", "ReportImages/CV2019_Annots_ElpsSoccer.csv")
zoom_image = report_augmented_image_and_annotations("ZOOM", soccer_image, "elps_soccer01_1056.png", "ReportImages/CV2019_Annots_ElpsSoccer.csv")
shift_image = report_augmented_image_and_annotations("SHIFT", soccer_image, "elps_soccer01_1056.png", "ReportImages/CV2019_Annots_ElpsSoccer.csv")

multiDisplay(["Original Image", "Horizontal Flipped Image", "Zoom In Image", "Shift Up Image"], [soccer_image, flip_image, zoom_image, shift_image], 2)
